In [1]:
from sklearn import svm, preprocessing, metrics
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# define useful functions

count = 0
RANGE = 1500
def closest_frame(df, f):
    for x in range(50):
        while len(face_data.loc[face_data['frame'] == f, 'milis'].values) == 0:
            f -= 1
    return f

def df_near(df, frame, ms_range=RANGE):
#     global count
#     count += len(frames_list)
    if frame is None:
        return None
    ms = df.loc[df['frame'] == closest_frame(df,frame), 'milis'].values[0]
#     locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms + ms_range)]
    locs = df.loc[(df['milis'] >= ms - ms_range) & (df['milis'] <= ms + ms_range)]
    return locs


def max_df_near(df, frames_list, ms_range=RANGE):
    d = df_near(df, frames_list)
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df_near(df, frames_list, ms_range=RANGE):
    d = df_near(df, frames_list)
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def max_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.max()).transpose()

def min_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.min()).transpose()

def mean_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.mean()).transpose()

def sd_df(d):
    if d is None:
        return None
    return pd.DataFrame(d.std()).transpose()

In [26]:
# gather data

def get_pid(filename):
    """ get pid from dirname """
    return filename.split('_')[0][1:]
def get_mode(filename):
    """ get mode from dirname """
    return int(file.split('_')[2][1])

up_data_max = []
down_data_max = []
up_data_min = []
down_data_min = []
all_data = []

up_data_sd = []
down_data_sd = []

up_data_mean = []
down_data_mean = []

all_data_max = []
all_data_min = []
all_data_mean = []
all_data_sd = []

PATH = "/Users/yoony/Desktop/si_research/face_analysis/implicit_feedback_si_faceanalysis/processed"
csvs = []
for file in sorted(os.listdir(PATH)):
    if file[0] == '.':
        continue
    print(file)
#     csvs.append(pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv')))
    data = pd.read_csv(os.path.join(PATH + "/" + file, 'webcam/aggregated_webcam.csv'))
    face_data = data.drop(columns=list(data.columns.values[2:678]))
    face_data = face_data.drop(columns=list(face_data.columns.values[19:]))
    
    signals = pd.read_csv('../my_buckets.csv')[['player_id', 'mode', 'signal_type', 'frame_number', 'code']]
    s1 = signals.loc[(signals['player_id'] == get_pid(file)) & (signals['mode'] == get_mode(file))]
    s_up = s1.loc[s1['signal_type'] == 'up'].frame_number.values
    s_down = s1.loc[s1['signal_type'] == 'down'].frame_number.values
    
    for x in s_up:
        up_locs = df_near(face_data, x)
#         print(len(up_locs))
        up_data_max.append(max_df(up_locs))
        up_data_min.append(min_df(up_locs))
        up_data_sd.append(sd_df(up_locs))
        up_data_mean.append(mean_df(up_locs))
        
        all_data_max.append(max_df(up_locs))
        all_data_min.append(min_df(up_locs))
        all_data_sd.append(sd_df(up_locs))
        all_data_mean.append(mean_df(up_locs))
        
    for x in s_down:
        down_locs = df_near(face_data, x)
        down_data_max.append(max_df(down_locs))
        down_data_min.append(min_df(down_locs))
        down_data_sd.append(sd_df(down_locs))
        down_data_mean.append(mean_df(down_locs))
        
        all_data_max.append(max_df(down_locs))
        all_data_min.append(min_df(down_locs))
        all_data_sd.append(sd_df(down_locs))
        all_data_mean.append(mean_df(down_locs))
        
    all_data.append(face_data)
    
    

P5ee6d2494fd04045b988a12d_voff_m1_g1_t2021_7_10_0_47
P5ee6d2494fd04045b988a12d_voff_m2_g2_t2021_7_10_0_50
P5ef33c05f4c5240413ef9ac2_voff_m1_g1_t2021_7_13_20_38
P5ef33c05f4c5240413ef9ac2_voff_m2_g2_t2021_7_13_20_34
P6081c511ded5945d9b70801e_voff_m1_g1_t2021_7_13_21_6
P6081c511ded5945d9b70801e_voff_m2_g2_t2021_7_13_21_11
P60d520539b147c4acaf44b77_voff_m1_g1_t2021_7_10_1_4
P60d520539b147c4acaf44b77_voff_m2_g2_t2021_7_10_1_1
P60e5e62659dc1912bc43ed35_voff_m1_g1_t2021_7_9_23_29
P60e5e62659dc1912bc43ed35_voff_m2_g2_t2021_7_9_23_24
P60e9e9618953d7a3430dd8f6_voff_m1_g1_t2021_7_13_18_51
P60e9e9618953d7a3430dd8f6_voff_m2_g2_t2021_7_13_18_57
P60edb7015a9231789057bdea_voff_m1_g1_t2021_7_13_21_7
P60edb7015a9231789057bdea_voff_m2_g2_t2021_7_13_21_11


In [5]:
def rename_columns(df, tag):
    a = df.copy()
    a.columns = list(map(lambda x: x + tag, a.columns))
    return a

def merge_dfs(dfs):
    a = dfs[0]
    for d in dfs[1:]:
        a = a.join(d, how='outer')
    return a
        

In [6]:
len(up_data_min)
len(down_data_min)

22

In [7]:
u1 = rename_columns(pd.concat(up_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
u2 = rename_columns(pd.concat(up_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
u3 = rename_columns(pd.concat(up_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
u4 = rename_columns(pd.concat(up_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [8]:
ups = merge_dfs([u1, u2, u3, u4])
ups['type'] = 'up'

In [9]:
d1 = rename_columns(pd.concat(down_data_max).drop(columns=['frame', 'milis']), '_max').reset_index(drop=True)
d2 = rename_columns(pd.concat(down_data_min).drop(columns=['frame', 'milis']), '_min').reset_index(drop=True)
d3 = rename_columns(pd.concat(down_data_mean).drop(columns=['frame', 'milis']), '_mean').reset_index(drop=True)
d4 = rename_columns(pd.concat(down_data_sd).drop(columns=['frame', 'milis']), '_sd').reset_index(drop=True)

In [10]:
downs = merge_dfs([d1, d2, d3, d4])
downs['type'] = 'down'

In [11]:
ups

,AU01_r_max,AU02_r_max,AU04_r_max,AU05_r_max,AU06_r_max,AU07_r_max,AU09_r_max,AU10_r_max,AU12_r_max,AU14_r_max,...,AU12_r_sd,AU14_r_sd,AU15_r_sd,AU17_r_sd,AU20_r_sd,AU23_r_sd,AU25_r_sd,AU26_r_sd,AU45_r_sd,type
0,1.96,1.28,0.70,0.74,0.84,0.96,2.66,0.82,0.27,0.73,...,0.070361,0.180347,0.000000,0.117914,0.041740,0.208949,0.251880,0.161573,0.000000,up
1,1.75,1.53,0.81,1.44,1.03,1.51,2.36,0.61,0.47,0.87,...,0.149772,0.243483,0.000000,0.164085,0.000000,0.169235,0.090149,0.078494,0.000000,up
2,1.25,1.60,0.87,1.51,1.68,1.49,2.52,0.62,0.33,1.27,...,0.112172,0.257599,0.000000,0.214771,0.189320,0.236253,0.000000,0.004472,0.131183,up
3,1.20,1.75,0.65,1.19,0.78,0.86,2.76,0.52,0.29,0.54,...,0.088299,0.096068,0.000000,0.138905,0.113698,0.250315,0.220221,0.000000,0.000000,up
4,1.42,1.57,0.74,1.17,0.70,1.18,2.24,0.82,0.38,0.87,...,0.091067,0.212280,0.000000,0.155514,0.000000,0.223830,0.268747,0.047474,0.000000,up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1.50,2.07,0.73,0.71,0.00,0.23,0.00,0.00,0.00,0.72,...,0.000000,0.183388,0.000000,0.000000,0.093140,0.031077,0.000000,0.016047,0.481694,up
60,1.46,1.60,0.91,0.41,0.00,0.20,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.041099,0.000000,0.000000,0.007454,0.000000,0.069745,0.147842,up
61,1.24,1.47,0.87,0.02,0.00,0.25,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.030742,0.000000,0.000000,0.000000,0.000000,0.102435,0.057915,up
62,1.52,1.88,0.91,0.05,0.00,0.50,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.641382,up


In [12]:
means = pd.DataFrame(pd.concat(up_data_sd).mean(), columns=['up_mean'])
means['down_mean'] = pd.concat(down_data_sd).mean()
means['mean_diff'] = means['up_mean'] - means['down_mean']
means['up_sd'] = pd.concat(up_data_sd).std()
means['down_sd'] = pd.concat(down_data_sd).std()
means['sd_diff'] = means['up_sd'] - means['down_sd']
means = means.drop(index=['frame', 'milis'])
# means['total'] = pd.concat(all_data).mean
# means.to_csv('face/au_data.csv')
means

,up_mean,down_mean,mean_diff,up_sd,down_sd,sd_diff
AU01_r,0.144969,0.121613,0.023356,0.115492,0.065975,0.049517
AU02_r,0.113001,0.075018,0.037983,0.111861,0.067516,0.044345
AU04_r,0.146317,0.128530,0.017787,0.070279,0.081587,-0.011309
AU05_r,0.226857,0.215854,0.011003,0.125739,0.055580,0.070159
AU06_r,0.047713,0.016162,0.031551,0.080890,0.035740,0.045149
AU07_r,0.100569,0.068672,0.031897,0.134931,0.078277,0.056654
AU09_r,0.089526,0.049740,0.039786,0.102431,0.060722,0.041709
AU10_r,0.069787,0.038495,0.031293,0.113324,0.052396,0.060928
AU12_r,0.035224,0.017604,0.017620,0.070150,0.041069,0.029082
AU14_r,0.100206,0.045478,0.054728,0.114204,0.079375,0.034830


In [13]:
clf = svm.SVC(kernel='linear')

In [14]:
#clean up and finalize the data
signal_data = pd.concat([ups, downs]).reset_index(drop=True)
signal_data
signal_types = signal_data['type'].values
signal_data = signal_data.drop(columns=['type'])

In [15]:
# split up into testing and training
X_train, X_test, y_train, y_test = train_test_split(signal_data, signal_types, test_size=0.3, random_state=109)

In [16]:
# normalize inputs
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler = preprocessing.StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [17]:
# fit data
clf.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [18]:
# predict
y_pred = clf.predict(X_test)

In [19]:
# Check accuracy
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
# print("Precision:",metrics.precision_score(y_test, y_pred))
# print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.8076923076923077


In [20]:
y_test

array(['up', 'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'up',
       'down', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'down', 'down',
       'up', 'up', 'up', 'up', 'up', 'up'], dtype=object)

In [21]:
y_pred

array(['up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up'], dtype=object)

In [22]:
metrics.f1_score(y_test, y_pred, average='weighted')
# metrics.f1_score(y_test, y_pred, pos_label='down')

0.7217675941080196